In [1]:
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
)

In [3]:
loader = TextLoader("./kpi_doc.txt",encoding = 'UTF-8')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
local_embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


c:\Users\Giamm\Desktop\RAG2\RAG5\langchainvenv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    Summarize the main themes in these retrieved docs: {docs}
    """ 
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = {"docs": format_docs}| prompt | model | StrOutputParser()

question = "what is kpi ontology?"

docs = vectorstore.similarity_search(query=question, k=4)

print(chain.invoke(docs))

content='Based on the retrieved doc, the main themes in the "KPIs Ontology" document appear to be:\n\n1. Classification and organization: The document describes a taxonomy that organizes Key Performance Indicators (KPIs) into different classes and subclasses, suggesting a structured approach to categorizing KPIs.\n2. Entity types: The ontology defines two fundamental entity types: KPI and Machine, indicating that the document aims to establish a clear distinction between these concepts.\n3. Subclasses: The document mentions that the KPI class branches into Utilization KPIs, suggesting that there may be other subclasses as well, adding complexity to the taxonomy.\n\nOverall, the document seems to focus on establishing a logical and structured framework for categorizing and understanding KPIs.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-11T23:50:56.1740702Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'to

In [92]:
import datetime
from typing import Optional
class KPIRequest(BaseModel):
    name: str = Field(description="The name of the KPI")    
    machine: str = Field(description="The machine the KPI is for")  
    aggregation: Optional[str] = Field(description="The aggregation type of the KPI")
    start_date: Optional[datetime.date] = Field(description="The start date of the KPI")
    end_date: Optional[datetime.date] = Field(description="The end date of the KPI")


In [54]:
import pydantic

In [ ]:
structured_llm = model.with_structured_output(KPIRequest)
#query_analyzer = prompt | structured_llm
try:
    response=structured_llm.invoke("""
                                    tell me what is the standard deviation of the energy consumption KPI 
                                   for the Large Capacity Cutting Machine 1 from 1st february 2020 to 1st march 2020
                                   """)
except pydantic.v1.error_wrappers.ValidationError as e:
    print(e)
#print(type(response.name))

In [100]:
print(response)

name='energy consumption KPI' machine='Large Capacity Cutting Machine 1' aggregation='stddev' start_date=datetime.date(2020, 2, 1) end_date=datetime.date(2020, 3, 1)


In [5]:
print(model.invoke("what is a cat?").content)

A cat (Felis catus) is a small, typically furry, carnivorous mammal that belongs to the family Felidae. Cats are one of the most popular pets in the world and have been domesticated for thousands of years.

Physical Characteristics:

* Cats have a slender body, usually with a long neck and tail.
* They have retractable claws, which they use for climbing, hunting, and self-defense.
* Their ears are pointed and can be rotated independently to pinpoint sounds.
* Cats have excellent eyesight, hearing, and sense of smell.
* They come in various coat colors, patterns, and lengths.

Behavioral Characteristics:

* Cats are known for their independence, aloofness, and playful personalities.
* They are highly territorial and have a strong instinct to mark their territory with urine, feces, or scratching marks.
* Cats are skilled hunters and can be solitary predators when necessary.
* They also have a strong social bond with humans, whom they often seek attention and affection from.

Domesticatio